# Preparação do *dataset* de trabalho

Este *notebook* tem como objetivo preparar uma parte específica (de escolha do usuário) para o *dataset* MAMEM SSVEP 

In [ ]:
# bloco apenas com organizações de imports
from google_drive_downloader import GoogleDriveDownloader as gdd
import numpy as np
import json

**Nota:** O arquivo `requirements.txt` foi adicionado à pasta do GitHub.
Para instalar todos os requerimentos necessários (no linux), basta fazer:

```
$ pip install -r requirements.txt
```

In [ ]:
# bloco de declaração dos identificadores de download (google drive) e setup
dataset_ids = {
    1: '1ym3JqI4_ZYNSBLxoF1vNxI5Hsgur_tha',
    2: '1tJ5tHbE-2jwr0gA33Gd873lRPao-e4dF',
    3: '1tXdpY-mser01POaP6Qwixny6LjsXXoXB',
    4: '1T00cliWp5yqqbbWZ7-rf2X4tOUQ-PvIQ',
    5: '1CYsBFMul9zB_vCy0XD7XVfKUl8vihDYZ',
    6: '1io6jdALBKeopELWXahPzuAi6EfYDgviW',
    7: '1YDkheRDPNDR1ujsqqC_SY6cebWHkw9Xt',
    8: '1jjoQJFDCi7O9Q-iaReAPpQnxC-HIKpQi',
}
label_id = '1mD5MXoh6tfQJFXIvdw2MQsEu6vZka6C0'
desc = '14kYNBZYdttqmSS_Vz6Bm_ztG9Uw1MC0y'

# ALTERE O ID DO DATASET DE SUA ESCOLHA AQUI ##################################
DS = 1

As funções de download abaixo não realizam o download mais de uma vez se o arquivo já existe.

In [ ]:
# download do stataset
gdd.download_file_from_google_drive(file_id=dataset_ids[DS],
                                    dest_path='files/data.npy',
                                    showsize=True)
# download do arquivo de marcações
gdd.download_file_from_google_drive(file_id=label_id,
                                    dest_path='files/labels.npy', showsize=True)

# download do arquivo de descrição
gdd.download_file_from_google_drive(file_id=desc,
                                    dest_path='files/descriptor.json',
                                    showsize=True)

In [4]:
# carregamento
X = np.load('files/data.npy')
y = np.load('files/labels.npy')
desc_file = open('files/descriptor.json')
descriptor = json.loads(desc_file.read())
desc_file.close()
print('Estruturas => dados', X.shape, 'labels', y.shape)

Estruturas => dados (125, 257, 1205) labels (125,)


Os dados estão estruturados em numpy arrays.

O "shape" `(125, 257, 1205)` para os dados EEG de `X`, significa que existem 125 *trials* (tradução aceitável: ensaio, julgamento), 257 *channels* que representam os eletrodos e 1205 dados que representam o potencial mensurado em ponto flutuante. Este é um array com **três dimensões**.

O "shape" `(125,)` para os dados de *labels* ou marcadores `y`, apresentam qual é o eletrodo respectivo a cada *trial*. Os *labels* estão com valores numéricos como é mostrado a seguir, o que facilita o processamento, mas não é intuitivo. 

In [5]:
y

array([4, 2, 3, 5, 1, 2, 5, 4, 2, 3, 1, 5, 4, 3, 2, 4, 1, 2, 5, 3, 4, 1,
       3, 1, 3, 4, 2, 3, 5, 1, 2, 5, 4, 2, 3, 1, 5, 4, 3, 2, 4, 1, 2, 5,
       3, 4, 1, 3, 1, 3, 4, 2, 3, 5, 1, 2, 5, 4, 2, 3, 1, 5, 4, 3, 2, 4,
       1, 2, 5, 3, 4, 1, 3, 1, 3, 4, 2, 3, 5, 1, 2, 5, 4, 2, 3, 1, 5, 4,
       3, 2, 4, 1, 2, 5, 3, 4, 1, 3, 1, 3, 4, 2, 3, 5, 1, 2, 5, 4, 2, 3,
       1, 5, 4, 3, 2, 4, 1, 2, 5, 3, 4, 1, 3, 1, 3], dtype=uint8)

Desta forma, foi preparado um arquivo de descrição no qual é possível saber qual é o rótulo correto do correspondente "número", além de outras informações como a taxa de amostragem (sampling rate), idade, tipo do cabelo (curto, médio ou comprido), sexo, tamanho da touca e se é destro ou canhoto. A princípio, imaginamos que estas informações não são tão importantes, mas problemas na experimentação podem ter culpa por um desses motivos, como por exemplo, o tamanho do cabelo. Veja os dados do voluntário escolhido:

In [6]:
print('Características do voluntário:', descriptor[str(DS)])
print('\nRótulos:', descriptor['frequencies'])
print('\nTaxa de amostragem:', descriptor['sampling_rate'])

Características do voluntário: {'age': '25', 'capsize': 'Adult Medium', 'gender': 'Male', 'hairtype': 'Regular', 'handedness': 'Right'}

Rótulos: {'1': 6.66, '2': 7.5, '3': 8.57, '4': 10.0, '5': 12.0}

Taxa de amostragem: 250


Como explicado em aula, apesar do autor do *dataset* apresentar a taxa de amostragem da captura dos dados igual a 250 Hz, o mesmo não foi verificado por meio dos dados. Os dados dos voluntários estão desbalanceados em quantidade, o que não nos permite criar um `numpy.array`, que permite apenas vetores não esparsos. Desta forma, teremos que recalcular a taxa de amostragem (*sample rate*)

In [7]:
# quantidade_de_dados / tempo_do_trial
descriptor['sampling_rate'] = X.shape[-1] / 5
print('Nova taxa de amostragem: {} Hz'.format(descriptor['sampling_rate']))

Nova taxa de amostragem: 241.0 Hz


## Montando um objeto estruturado EEG do *dataset*

Neste ponto, iremos trabalhar com a biblioteca MNE. Ela nos permite além de estruturar melhor os dados:
* Realizar o mapeamento dos eletrodos de acordo com o equipamento utilizado;
* Filtrar canais de acordo com a necessidade do experimento;
* Aplicar filtros temporais e espaciais;
* Mostrar *plots* dinâmicos
* Aplicar algoritmos de extração e seleção de características, entre outros...

### Tarefas

- Estudar a biblioteca MNE disponível [site oficial](https://mne.tools/stable/index.html)
    - Um bom começo é ver por meio dos exemplos ["Criando objetos MNE a partir de dados em vetores"](https://mne.tools/stable/auto_examples/io/plot_objects_from_arrays.html#sphx-glr-auto-examples-io-plot-objects-from-arrays-py)
- Ler o documento sobre o dataset de trabalho disponível [neste link](https://figshare.com/articles/MAMEM_EEG_SSVEP_Dataset_II_256_channels_11_subjects_5_frequencies_presented_simultaneously_/3153409) e obter algumas informações importantes como:
    - Qual aparelho de EEG é utilizado;
    - Se filtros temporais (*bandpass*, *lowpass*, *highpass*) já foram aplicados;
    - Qual é a configuração da touca, que será utilizada para o setup do parâmetro `mount` no metadado de informações.
    - Verificar se as características dos voluntários influenciaram no experimento;
    - Entender a configuração do ambiente de experimento;
    - além de outras informações relevantes para a nossa análise...